In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/study/16715robotsim/project/Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using OrdinaryDiffEq
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Test
using Test
using Plots
using Convex, SCS
using JLD

## System properties

In [3]:

const g = 9.81

const m = 1
const μ = 0.5

const w = 1.0 # width of the box
const h = 0.8 # height of the box
const M = [m 0 0; 0 m 0; 0 0 m*((h/2)^2+(w/2)^2)/3]

# set of contact modes
const modes = [0 0 0 0; # both free
        1 0 0 0; # sticking
        1 0 -1 0; # left-slide
        1 0 1 0; # right-slide
        0 1 0 0;
        0 1 0 -1;
        0 1 0 1;
        1 1 0 0;
        1 1 -1 -1;
        1 1 1 1]

const n_contacts = 2


2

In [4]:
# simulation step size
const Δt = 0.05
# tolerance for contacts
const tol_c = 1e-5

1.0e-5

## Simulation functions

In [5]:
function R_2D(θ)
    R = [cos(θ) -sin(θ); sin(θ) cos(θ)]
    return R
end

R_2D (generic function with 1 method)

In [6]:
# constraints/contacts
function compute_a(q)
    y = q[2]
    θ = q[3]
    a1 = y - 0.5*h*cos(θ) - 0.5*w*sin(θ)
    a2 = y - 0.5*h*cos(θ) + 0.5*w*sin(θ)
    return [a1; a2]
end

function compute_a_t(q)
    p1 = R_2D(q[3])*[-w/2; -h/2] 
    p2 = R_2D(q[3])*[w/2; -h/2] 
    at1 = p1[1] + q[1]
    at2 = p2[1] + q[1]
    return [at1; at2]
end

# constraints jacobian
function compute_A(q)
    θ = q[3]
    A = [0 1 0.5*h*sin(θ)-0.5*w*cos(θ); 0 1 0.5*h*sin(θ)+0.5*w*cos(θ)]
    return A
end

function compute_dA(q, dq)
    dA = reshape(ForwardDiff.jacobian(compute_A, q)*dq, n_contacts, 3)
    return dA
end

function compute_A_tangent(q)
    A_t = ForwardDiff.jacobian(compute_a_t,q)
    return A_t
end

function compute_dA_tangent(q, dq)
    dA_t = reshape(ForwardDiff.jacobian(compute_A_tangent, q)*dq, n_contacts, 3)
    return dA_t
end

compute_dA_tangent (generic function with 1 method)

In [7]:
function contact_mode_constraints(x, contactMode)
    
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    A_all = zeros(0,3)
    A_all_f = zeros(0,3)
    dA_all = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            A_all_f = [A_all_f; A[k,:]'; A_t[k,:]']
            dA_all = [dA_all; dA[k,:]'; dA_t[k,:]']
            A_all = [A_all; A[k,:]'; A_t[k,:]']
        else
            A_all_f = [A_all_f; A[k,:]'-ss*μ*A_t[k,:]']
            dA_all = [dA_all; dA[k,:]']
            A_all = [A_all; A[k,:]']
        end
    end
    
    return A_all_f, A_all, dA_all
end

contact_mode_constraints (generic function with 1 method)

In [8]:
function contact_force_constraints(λ, contactMode)
    # c > 0
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    ss_active = ss_mode[cs_mode]
    
    i = 1
    ic = 1
    c = zeros(sum(ss_active.==0)*3 + sum(ss_active.!=0))
    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            c[ic:ic+2] = [-λ[i]; -μ*λ[i] - λ[i+1]; -μ*λ[i] + λ[i+1]]
            ic += 3
            i += 2
        else
            c[ic] = -λ[i]
            i += 1
            ic += 1
        end
    end
    return c
end

contact_force_constraints (generic function with 1 method)

In [9]:
function sliding_velocity_constraints(x, contactMode)
    # reture A_eq, A_geq, A_eq*dq = 0, A_geq*dq >= 0
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    AA_eq = zeros(0,3)
    AA_geq = zeros(0,3)
    dAA_eq = zeros(0,3)
    dAA_geq = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            AA_eq = [AA_eq; A[k,:]'; A_t[k,:]']
            dAA_eq = [dAA_eq; dA[k,:]'; dA_t[k,:]']
        else
            AA_eq = [AA_eq; A[k,:]']
            dAA_eq = [dAA_eq; dA[k,:]']
            AA_geq = [AA_geq; ss*A_t[k,:]']
            dAA_geq = [dAA_geq; ss*dA_t[k,:]']
        end
    end
    
    return AA_eq, AA_geq, dAA_eq, dAA_geq
end

sliding_velocity_constraints (generic function with 1 method)

In [175]:
function contact_tangent_velocity_constraints(x, contactMode)
    # reture A_eq, A_geq, A_eq*dq = 0, A_geq*dq >= 0
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    AA_eq = zeros(0,3)
    AA_geq = zeros(0,3)
    dAA_eq = zeros(0,3)
    dAA_geq = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            AA_eq = [AA_eq; A_t[k,:]']
            dAA_eq = [dAA_eq; dA_t[k,:]']
        else
            AA_geq = [AA_geq; ss*A_t[k,:]']
            dAA_geq = [dAA_geq; ss*dA_t[k,:]']
        end
    end
    
    return AA_eq, AA_geq, dAA_eq, dAA_geq
end

contact_tangent_velocity_constraints (generic function with 1 method)

In [10]:
function solveEOM(x, contactMode, u_control)
    # contactMode: bool vector, indicates which constraints are active
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, contactMode)
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    Y = zeros(3)
    Y .= u_control
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    #z = blockMat\b
    #println(blockMat)
    if rank(blockMat) < length(b)
#         z =pinv(blockMat)*b
        H = [zeros(3,size(blockMat,2)); zeros(size(blockMat,1)-3,3) 1e-6*I]
        z =(blockMat.+H)\b
    else
        z = blockMat\b
    end
    
    ddq = z[1:3]
    if (sum(contactMode[1:n_contacts])>=1)
        λ = z[4:end]
    else
        λ = []
    end
    
    return ddq, λ
end

solveEOM (generic function with 1 method)

In [183]:
function computeResetMap(x, contactMode)
    q = x[1:3]
    dq = x[4:6]

    A_f, A, dA = contact_mode_constraints(x, contactMode)
    
    c = size(A, 1)
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 
    
    if rank(blockMat) < 3+c
        z = pinv(blockMat)*[M*dq; zeros(c)]
        p_hat = z[4:end]
        dq_p = z[1:3]
#         dq_p = z[1:3] .+ A\(-A*z[1:3])
    else
        z = blockMat\[M*dq; zeros(c)]
        dq_p = z[1:3]
        p_hat = z[4:end]
    end
        

    return dq_p, p_hat
end

computeResetMap (generic function with 1 method)

In [375]:
function compute_FA(x, u_control)
    
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    
    active_cs = abs.(a) .< tol_c
    inactive_cs = abs.(a) .> tol_c
    
    if sum(active_cs) == 0
        return zeros(4)
    end
    
    possibleModes = zeros(Int, 0, size(modes,2))
    
    contactMode = zeros(n_contacts*2)
    
    for k = 1:size(modes,1)
        m_cs = modes[k, 1:n_contacts].==1
        if length(findall(z->z==true, m_cs[inactive_cs])) == 0
            possibleModes = [possibleModes; modes[k, :]']
        end
    end

    max_cons = 0
    
    
    for kk = 1:size(possibleModes, 1)      
        
        m = possibleModes[kk,:]
        
        separate_cs = (m[1:n_contacts].!=1) .& active_cs
        _, A_separate, dA_separate = contact_mode_constraints(x, [separate_cs; ones(n_contacts)]) 

        ddq, λ = solveEOM(x, m, u_control)
        
        c_λ = contact_force_constraints(λ, m)
        
        if all(c_λ.>=0)
        
            As_eq, As_geq, dAs_eq, dAs_geq = sliding_velocity_constraints(x, m)

            sep_vel_cond = ((A_separate*dq).>0) .| ((A_separate*ddq .+ dA_separate*dq).>=0)
            maintain_vel_cond = all(abs.(As_eq*dq).<tol_c) & all((As_geq*dq).>tol_c)

            if ~maintain_vel_cond
                if any((As_geq*dq).<tol_c)
                    maintain_vel_cond = all(abs.(dAs_eq*dq + As_eq*ddq).<tol_c) & all((dAs_geq*dq + As_geq*ddq).>0)
                end
            end

            if all(c_λ.>=0) && all(sep_vel_cond) && maintain_vel_cond
    
                if sum(m[1:n_contacts]) > max_cons
                    contactMode = m
                    max_cons = sum(m[1:n_contacts])
                end
            end
        end
    end
    
    return contactMode
end

compute_FA (generic function with 1 method)

In [377]:
function compute_IV(x)
    
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    
    active_cs = abs.(a) .< tol_c
    inactive_cs = abs.(a) .> tol_c
    
    if sum(active_cs) == 0
        return zeros(Int, 4)
    end
    
    possibleModes = zeros(Bool, 0, size(modes,2))
    
    contactMode = zeros(n_contacts*2)
    
    for k = 1:size(modes,1)
        m_cs = modes[k, 1:n_contacts].==1
        if length(findall(z->z==true, m_cs[inactive_cs])) == 0
            possibleModes = [possibleModes; modes[k, :]']
        end
    end
    
    max_cons = 0
    
    for kk = 1:size(possibleModes, 1)
        
        m = possibleModes[kk,:]
        
        separate_cs = (m[1:n_contacts].!=1) .& active_cs
        _, A_separate, _ = contact_mode_constraints(x, [separate_cs; ones(n_contacts)]) 

        dq_p, p_hat = computeResetMap(x, m)
        
        c_p_hat = contact_force_constraints(p_hat, m)
        
        As_eq, As_geq, _, _ = sliding_velocity_constraints(x, m)
        
        if all(c_p_hat.>=0) && all(A_separate*dq_p.>=0) && all(abs.(As_eq*dq_p).<tol_c) && all(As_geq*dq_p.>0)
            if sum(m[1:n_contacts]) > max_cons
                contactMode = m
                max_cons = sum(m[1:n_contacts])
            end
        end
    end
    
    return contactMode
end

compute_IV (generic function with 1 method)

In [255]:
function mode(x, u_control)
    return Vector{Int}(compute_FA(x, u_control))
end

mode (generic function with 1 method)

In [291]:
function guard_conditions(x, contactMode, u_control)
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
#     a[contactMode[1:n_contacts] .== 1] .= 0.0
    
    # for sliding->sticking transition
    v_all = zeros(n_contacts)
#     _, As_geq, _, _ = sliding_velocity_constraints(x, contactMode)
#     v_all[1:size(As_geq,1)] = -As_geq*dq
    At_eq, At_geq, _, _ = contact_tangent_velocity_constraints(x, contactMode)
    v_all[1:size(At_geq,1)] = -At_geq*dq
    
    ddq, λ = solveEOM(x, contactMode, u_control)
    c_λ = contact_force_constraints(λ, contactMode)
    c_λ_all = zeros(3*n_contacts)
    c_λ_all[1:length(c_λ)] = c_λ
    
    c = [a; v_all; c_λ_all]
    if debug == true     
#         println("guard_conditions ", c)
    end
    
    dir = [-ones(Int,length(a)); ones(Int,length(v_all)); ones(Int,length(c_λ_all))]
    
    return c, dir
end

guard_conditions (generic function with 1 method)

## Wrappers for ODE solver

In [366]:
function dynamics!(dx, x, p, t)
    # p from integrator: (contact mode, controller, t_control, h_control, u_control)
    
    if debug == true
        println("Dynamics evalutation ", t, "x ", x)
    end
    
    # bound the simulation
    if any(abs.(x[1:3]).>10) || any(abs.(x).>50) || (x[2] < -0.2)
        dx .= [x[4:6]; zeros(3)]
        return
    end

    q = x[1:3]
    dq = x[4:6]
    contactMode = p[1]
    controller = p[2]
    t_control = p[3][1]
    h_control = p[4]
    u_control = zeros(3)
    u_control .= p[5]
    
    if t > (t_control + h_control)
        
        p[3] .= [Float64(t)]
        u_control = controller(x, contactMode)
        p[5] .= u_control
        if debug == true
            println("Control evaluation ", u_control)
        end
    end
    
    ddq, λ = solveEOM(x, contactMode, u_control)
    
    #println("acceleration ", ddq)
    
    dx .= [dq; ddq]
    c_λ = contact_force_constraints(λ, contactMode)
    if(sum(c_λ.<-tol_c) > 0)
#         println(c_λ)
        new_contactMode = compute_FA(x, u_control)
        if (all(new_contactMode .==0))
            return
        end
        p[1] .= reshape(new_contactMode,size(p[1]))
        if debug == true
            println("New mode from constraints ", new_contactMode)
            println(λ)
        end
    end
end

dynamics! (generic function with 1 method)

In [345]:
function conditions(out, x, t, integrator)
    contactMode = integrator.p[1]
    u_control = integrator.p[5]
    c, dir = guard_conditions(x, contactMode, u_control)
    out .= c
end

function affect!(integrator, idx)
    if debug == true
        println("upper crossing.")
    end
    contactMode = integrator.p[1]
    x = integrator.u
    u_control = integrator.p[5]
    c, dir = guard_conditions(x, contactMode, u_control)
    
    # only consider upcrossing forces and constraints values(FA comp)
    # forces
    if dir[idx] > 0
        new_contactMode = compute_FA(x, u_control)
        controller = integrator.p[2]
        u_control = controller(x, new_contactMode)
        integrator.p[5] .= u_control
        if (eltype(new_contactMode)==Float64)
            return
        end
        integrator.p[1] .= reshape(new_contactMode,size(integrator.p[1]))
        if debug == true
            println("New mode from FA: ", new_contactMode)
            println(x, u_control)
        end
    end
    # constraints
#     if dir[idx] < 0
#         new_contactMode = contactMode
#         new_contactMode[idx] = false
#     end
end

function affect_neg!(integrator, idx)
    if debug == true
        println("down crossing.")
    end
    contactMode = integrator.p[1]
    x = integrator.u
    u_control = integrator.p[5]
    c, dir = guard_conditions(x, contactMode, u_control)
    
    # only consider down crossing constraint value(IV comp)
    if dir[idx] < 0
        new_contactMode = compute_IV(x)
        if (all(new_contactMode .==0))
            return
        end
        controller = integrator.p[2]
        u_control = controller(x, new_contactMode)
        integrator.p[5] .= u_control
        
        dq_p, p_hat = computeResetMap(x, new_contactMode)
        integrator.u .= [x[1:3]; dq_p]
        integrator.p[1] .= reshape(new_contactMode,size(integrator.p[1]))
        
        if debug == true
            println("New mode from IV: ", new_contactMode)
            println(x)
        end

    end
end

affect_neg! (generic function with 1 method)

## goal specific conditions

In [19]:
# reference conditions
xref = [0;h/2;0;1;0;0]
uref = [μ*m*g;0]
nominal_mode = [1 1 1 1]

# pusher location
pusher_p = [-w/2;0]

2-element Vector{Float64}:
 -0.5
  0.0

## Basic controllers

In [20]:
function no_controller(x, contactMode)
    return zeros(3)
end

function force_controller(x, contactMode)
    f = zeros(3)
    f[1] = 2*μ*m*g
    return f
end

function pd_controller(x, contactMode)
    # for sliding velocity
    q = x[1:3]
    dq = x[4:6]
    k = 10
    d = 1
    eq = q - xref[1:3]
    eq[1] = 0
    f = -k.*eq .- d.*(dq - xref[4:6])
    
    # add sliding force compensation
    if abs(f[1]) > 1e-3
        f[1] += μ*m*g
    end
    return f
end

function pd_pusher_controller(x, contactMode)
    # for sliding velocity
    q = x[1:3]
    dq = x[4:6]
    
    k = 10
    d = 1
    
    A = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    
    eq = q - xref[1:3]
    eq[1] = 0
    fd = -k.*eq .- d.*(dq - xref[4:6])
    
    # add sliding force compensation
    if abs(fd[1]) > 1e-3
        fd[1] += μ*m*g
    end
    
    c = A\fd
    f = A*c
    return f
end
    

pd_pusher_controller (generic function with 1 method)

## LQR 
Push the box with a pusher (a point contact)

In [21]:
"""
    riccati(A,B,Q,R,Qf,N)

Use backward riccati recursion to solve the finite-horizon time-invariant LQR problem.
Returns vectors of the feedback gains `K` and cost-to-go matrices `P`, where `length(K) == N-1`,
`length(P) == N`, and `size(K[i]) == (m,n)` and `size(P[i]) == (n,n)`.

# Arguments:
* `A`: `(n,n)` discrete dynamics Jacobian wrt the state
* `B`: `(n,m)` discrete dynamics Jacobian wrt the control
* `Q`: `(n,n)` stage-wise cost matrix for states
* `R`: `(m,m)` stage-wise cost matrix for controls
* `Qf`: `(n,n)` cost matrix for terminal state
* `N`: Integer number of time steps (horizon length).
"""
function riccati(A,B,Q,R,Qf,N)
    # initialize the output
    n,m = size(B)
    P = [zeros(n,n) for k = 1:N]
    K = [zeros(m,n) for k = 1:N-1]
    
    P[end] .= Qf
    for k = reverse(1:N-1) 
        K[k] .= (R + B'P[k+1]*B)\(B'P[k+1]*A)
        P[k] .= Q + A'P[k+1]*A - A'P[k+1]*B*K[k]
    end
    
    # return the feedback gains and ctg matrices
    return K,P
end

riccati

In [22]:
function rk4_step(f,xk,u)
    # classic rk4 parameters
    a21 = 0.5
    a31 = 0
    a32 = 0.5
    a41 = 0
    a42 = 0
    a43 = 1
    b1 = 1/6
    b2 = 1/3
    b3 = 1/3
    b4 = 1/6
    
    f1 = f(xk,u)
    f2 = f(xk + Δt*a21*f1,u)
    f3 = f(xk + Δt*a31*f1 + Δt*a32*f2,u)
    f4 = f(xk + Δt*a41*f1 + Δt*a42*f2 + Δt*a43*f3,u)

    xn = xk + Δt*(b1*f1 + b2*f2 + b3*f3 + b4*f4)
    
    return xn
end

function continuous_dynamics(x, u, mode)
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, mode)
    
    # contact constraints
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    Y = Jc*u
    
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    if rank(blockMat) < length(b)
        z =pinv(blockMat)*b
    else
        z = blockMat\b
    end

    ddq = z[1:3]
    
    return [dq;ddq]
end

function continuous_dynamics_differentiable(x, u, mode)
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, mode)
    
    # contact constraints
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    Y = Jc*u
    
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    H = [zeros(3,size(blockMat,2)); zeros(size(blockMat,1)-3,3) 1e-6*I]
    z =(blockMat.+H)\b


    ddq = z[1:3]
    
    return [dq;ddq]
end

function discrete_dynamics(x, u, mode)
    xn = rk4_step((_x,_u)->continuous_dynamics_differentiable(_x,_u,mode),x,u)
    return xn
end

discrete_dynamics (generic function with 1 method)

In [23]:
# compute infinite horizon K
A_lqr = ForwardDiff.jacobian(_x->discrete_dynamics(_x, uref, nominal_mode), xref)
B_lqr = ForwardDiff.jacobian(_u->discrete_dynamics(xref, _u, nominal_mode), uref)
Q_lqr = Diagonal([0;1.0;1.0;1.0;0.1;1.0])
R_lqr = Diagonal([0.1;0.1])
Ks, _ = riccati(A_lqr,B_lqr,Q_lqr,R_lqr,Q_lqr,50)
K_lqr = Ks[1]

2×6 Matrix{Float64}:
 0.0   4.88068e-6  -5.22899e-6  2.58945   8.85062e-6  -1.5144e-5
 0.0  -1.25659e-5   1.53925e-5  1.29473  -2.16825e-5   3.69867e-5

In [87]:
function pusher_lqr_controller(x, contactMode)
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    u = uref .- K_lqr*(x .- xref)
    u[u .> 20] .= 20
    u[u .< -20] .= -20
    f = Jc*u
    return f
end

pusher_lqr_controller (generic function with 1 method)

## Hybrid controller

### Hybrid system definition

In [83]:
# domain
# ineqs > 0; eqs = 0
function domain(x, contactMode)
    
    cs = contactMode[1:n_contacts] .== 1
    ss = contactMode[n_contacts+1:end]
    
    q = x[1:3]
    dq = x[4:6]
    
    # a(q) = or > 0
    
    a = compute_a(q)
    
    eqs_a = a[cs]
    ineqs_a = a[.~cs]
    
    # A_eq*dq = 0; A_geq*dq > 0
    
    A_eq, A_geq, _, _ = sliding_velocity_constraints(x, contactMode)
    
    eqs_A = A_eq*dq
    ineqs_A = A_geq*dq
    
    # separating contacts
    separate_cs = (cs.!=1) .& (abs.(a) .< tol_c)
    _, A_separate, _ = contact_mode_constraints(x, [separate_cs; ones(n_contacts)])
    
    ineqs_A_sep = A_separate*dq
    
    ineqs = [zeros(0); ineqs_a; ineqs_A]#; ineqs_A_sep]
    eqs = [zeros(0); eqs_a; eqs_A]
    
    return ineqs, eqs
end

domain (generic function with 1 method)

In [26]:
function guard_set(x, mode_from, mode_to)
    
    ineqs = zeros(0)
    eqs = zeros(0)
    
    q = x[1:3]
    dq = x[4:6]
    
    d_ineqs, d_eqs = domain(x, mode_from)
    # c: d_ineqs > 0, d_eqs = 0
    ineqs = [ineqs; d_ineqs]
    eqs = [eqs; d_eqs]
    
    cs_mode_to = mode_to[1:n_contacts].==1
    cs_mode_from = mode_from[1:n_contacts].==1
    
    A_f_new, A_new, dA_new = contact_mode_constraints(x, mode_to)
    
    # if there is new impact
    new_cs = (cs_mode_from.==false).& (cs_mode_to.==true)
    if sum(new_cs) > 0
            a_new = compute_a(x)
        # c: a_new[new_cs] .== 0
        eqs = [eqs; a_new[new_cs]]
        
        _, A_impact, _, = contact_mode_constraints(x, [new_cs; ones(n_contacts)])
        #c: A_impact*dq < 0
        ineqs = [ineqs; -A_impact*dq]
        
        p_hat = pinv(A_new*inv(M)*A_f_new')*A_new*dq
        #c: contact_force_constraints(p_hat, mode_to) > 0
        ineqs = [ineqs; contact_force_constraints(p_hat, mode_to)]
    end
    
    # if it is stick -> slide: do nothing, guard = domain
    # if it is slide -> stick: A_stick dq = 0
    ss_mode_to = mode_to[n_contacts+1:end]
    ss_mode_from = mode_from[n_contacts+1:end]
    ss_new_stick = (cs_mode_from.==true).& (cs_mode_to.==true) .& (ss_mode_to .== 0) .& (ss_mode_from .!=0)
    
    if sum(ss_new_stick) > 0    
        A_stick, _, _, _ = sliding_velocity_constraints(x, [ss_new_stick; zeros(n_contacts)])
        # c: A_stick*dq = 0
        eqs = [eqs; A_stick*dq]
    end
        
    return ineqs, eqs
end

guard_set (generic function with 1 method)

In [27]:
function jumpmap(x, mode_from, mode_to)
    
    q = x[1:3]
    dq = x[4:6]

    A_f, A, dA = contact_mode_constraints(x, mode_to)
    
    c = size(A, 1)
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 
    
    b = [M*dq; zeros(c)]

    H = [zeros(3,size(blockMat,2)); zeros(size(blockMat,1)-3,3) 1e-6*I]
    z =(blockMat.+H)\b
        
    dq_p = z[1:3]
    p_hat = z[4:end]
    
    return [x[1:3]; dq_p]
end

jumpmap (generic function with 1 method)

In [28]:
function V(x)
    q = x[1:3] + x[4:6]
    dq = x[4:6]
    v = 0.5*([q;dq] .- xref)'*Q_lqr*([q;dq] .- xref)
    return v
end 

function dVdx(x)
    res = ForwardDiff.gradient(V, x)
    return res
end

dVdx (generic function with 1 method)

In [29]:
function hybrid_constraints_matrix(x, u_ref)
    # A*z + b>= 0
    # u = z[1:2]
    # β = z[3:175]
    # α = z[176:end]
    
    n = 2+173+19
    m = 19 
    A = zeros(m, n)
    b = zeros(m)

    # α = 10
    
    Vx = dVdx(x)
    Vv = V(x)
    
    n_modes = size(modes,1)
    
    β_idx = 1
    
    nominal_mode = modes[10,:] # [1 1 1 1]
    

    # mode dynamics
    
    for k = 1:n_modes
    
        m = modes[k,:]
        
        d_ineq, d_eq = domain(x, m)
        
        # TODO: debug this!!!
        if sum(d_ineq.<0)>0 || sum(abs.(d_eq) .> tol_c)>0
            b[k] = 0.1
            continue
        end
        
        n_ineq = size(d_ineq, 1)
        n_eq = size(d_eq, 1)
        
        dfdu = ForwardDiff.jacobian(_u->continuous_dynamics_differentiable(x, _u, m), u_ref)
        
        b[k] = -Vx'*continuous_dynamics_differentiable(x, u_ref, m)
        # α
        A[k, 175+k] = -Vv
        A[k, 1:2] = -Vx'*dfdu
        A[k, β_idx:β_idx+n_ineq-1] .= -d_ineq
        A[k, β_idx+n_ineq:β_idx+n_ineq+n_eq-1] .= -d_eq 
        A[k, β_idx+n_ineq+n_eq:β_idx+n_ineq+2*n_eq-1] .= d_eq 
        
        β_idx += n_ineq + 2*n_eq
        
    end
    
    nominal_mode = modes[10,:]
    
    for k = 1:(n_modes-1)
        
        mode_from = modes[k,:]
        
        d_ineq, d_eq = guard_set(x, mode_from, nominal_mode)
        
        if sum(d_ineq.<0)>0 || sum(abs.(d_eq) .> tol_c)>0
            b[k] = 0.1
            continue
        end
        
        n_ineq = size(d_ineq, 1)
        n_eq = size(d_eq, 1)
        
        xp = jumpmap(x, mode_from, nominal_mode)
        
        dfdu = ForwardDiff.jacobian(_u->continuous_dynamics_differentiable(xp, _u, nominal_mode), u_ref)
        
        b[n_modes+k] = -Vx'*continuous_dynamics_differentiable(xp, u_ref, nominal_mode)
        A[k, 175+n_modes+k] = -Vv
        A[n_modes+k, 1:2] = -Vx'*dfdu
        A[n_modes+k, β_idx:β_idx+n_ineq-1] .= -d_ineq
        A[n_modes+k, β_idx+n_ineq:β_idx+n_ineq+n_eq-1] .= -d_eq 
        A[n_modes+k, β_idx+n_ineq+n_eq:β_idx+n_ineq+2*n_eq-1] .= d_eq 
        
        β_idx += n_ineq + 2*n_eq
        
    end
    
    return A, b
    
end

hybrid_constraints_matrix (generic function with 1 method)

### Interface with Convex.jl

In [ ]:
# n_control_var = 2+173+19
# n_control_constraints = 19 # total number of constraints

### define the controller

In [215]:
function hybrid_reference_lqr(x, contactMode)
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    u = uref .- K_lqr*(x .- xref)
    u[u .> 20] .= 20
    u[u .< -20] .= -20
    return u
end

hybrid_reference_lqr (generic function with 1 method)

In [382]:
function hybrid_controller(x, contactMode)
    
    u_ref = hybrid_controller_reference(x, contactMode) # from reference controller
    
    α_ref = 10

    z = Variable(194)
    A, b = hybrid_constraints_matrix(x, u_ref)
    
    μ_mnp = 0.8
    FC = [1 0; μ_mnp -1; μ_mnp 1]

    problem = minimize(sumsquares(z - [zeros(175); α_ref*ones(19)]))
    problem.constraints += A*z + b >= 0
    problem.constraints += z[3:end] >= 0
    #problem.constraints += FC*(z[1:2]+u_ref) >= 0
    problem.constraints += z[1:2] <= 20
    problem.constraints += z[1:2] >= -20


    Convex.solve!(problem, () -> SCS.Optimizer(verbose=false))
    
    z_sol = evaluate(z)
    if any(isnan.(z_sol)) || sum(sol) == Inf # infeasible
        z_sol = zeros(175)
#         println("Infeasible: ")
#         println(x)
#         println(contactMode)
    end
    
    u = z_sol[1:2] .+ u_ref
    
    # return space wrench
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    f = Jc*u
    
    return f
    
end

hybrid_controller (generic function with 1 method)

## simulate the sliding box 

In [ ]:
# specify the controller
controller = lqr_controller
hybrid_controller_reference = hybrid_reference_lqr

h_control = Δt

In [ ]:
# initial condition

# # basic test
# x0 = [0;h/2;0;0;0;0]

# # lqr fail 1
# θ0 = -0.3
# x0 = [0;h/2 - sin(θ0)*w/2;θ0;0;0;0]

# # lqr success 2
# θ0 = -0.3
# x0 = [0;h/2 - sin(θ0)*w/2 + 0.2;θ0;0;0;0]

# # lqr fail 3
# θ0 = -0.5
# x0 = [0;h/2 - sin(θ0)*w/2;θ0;0;0;0]

# # lqr fail 4
# θ0 = -0.4
# x0 = [0;h/2 - sin(θ0)*w/2;θ0;0;0;0]

# lqr fail 5
θ0 = -0.4
x0 = [0;h/2 - sin(θ0)*w/2 + 0.5;θ0;0;0;0]

In [ ]:
debug = true
initial_mode = mode(x0, controller(x0))
# initial_mode = [1 1 0 0]
tspan = (0.0, 3.0)
callback_length = 5*n_contacts

prob = ODEProblem(dynamics!, x0, tspan, (initial_mode, controller, [0.0], h_control, zeros(3)))
cb = VectorContinuousCallback(conditions, affect!, affect_neg!, callback_length)
sol = solve(prob, Tsit5(); callback = cb, abstol=1e-15,reltol=1e-15, adaptive=false,dt=Δt)
# sol = solve(prob, Tsit5(); callback = cb, abstol=1e-9,reltol=1e-9, adaptive=false,dt=Δt)
# sol = solve(prob, Tsit5(); callback = cb, adaptive=false,dt=Δt)
println("Simulation status: ", sol.retcode)

In [ ]:
function boxshape(q)
    p1 = q[1:2] + R_2D(q[3])*[w/2;h/2]
    p2 = q[1:2] + R_2D(q[3])*[w/2;-h/2]
    p3 = q[1:2] + R_2D(q[3])*[-w/2;-h/2]
    p4 = q[1:2] + R_2D(q[3])*[-w/2;h/2]
    pp = [p1 p2 p3 p4]
    return Shape(pp[1,:], pp[2,:])
end

In [ ]:
# animation 
n = length(sol.t)
# n = Int(floor(0.77/Δt))
x = zeros(n)
y = zeros(n)
θ = zeros(n)
for k = 1:n
    x[k] = sol.u[k][1]
    y[k] = sol.u[k][2]
    θ[k] = sol.u[k][3]
end
anim = @animate for i ∈ 1:n
    plot([-10,5],[0,0], lw = 2, c=:black, xlims=(-3,3), ylims=(-0.5,3))
    plot!(boxshape([x[i],y[i],θ[i]]), aspect_ratio=:equal, c=:gray, opacity=.5, legend=false)
end

In [ ]:
gif(anim, "anim.gif", fps = Int(floor(n/tspan[2])));

## Compute Contration Region (Monte-carlo)

In [50]:
function boxshape(q)
    p1 = q[1:2] + R_2D(q[3])*[w/2;h/2]
    p2 = q[1:2] + R_2D(q[3])*[w/2;-h/2]
    p3 = q[1:2] + R_2D(q[3])*[-w/2;-h/2]
    p4 = q[1:2] + R_2D(q[3])*[-w/2;h/2]
    pp = [p1 p2 p3 p4]
    return Shape(pp[1,:], pp[2,:])
end

boxshape (generic function with 1 method)

In [388]:
hybrid_controller_reference = hybrid_reference_lqr
tspan = (0.0, 2.5)
callback_length = 5*n_contacts
h_control = Δt

0.05

In [33]:
N = 31

31

In [34]:
y_l = h/2
y_u = h/2 + 0.8
θ_l = -pi/4
θ_u = pi/4
δy = (y_u - y_l)/(N-1)
δθ = (θ_u - θ_l)/(N-1)
println(δy," ", (180/pi)*δθ)

0.026666666666666672 3.0


In [211]:
ctrl = "hybrid"

"hybrid"

In [212]:
if ctrl == "lqr"
    print("lqr")
    controller = pusher_lqr_controller
    errs = zeros(N*N);
else 
    print("hybrid")
    controller = hybrid_controller
    errs = zeros(N*N);
end

hybrid

961×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 ⋮    
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

In [277]:
controller

hybrid_controller (generic function with 1 method)

In [138]:
debug = false

false

In [278]:
k = 1
q0 = zeros(N*N,2)
for ky = 0:(N-1), kθ = 0:(N-1)
    print("k ", k, "ky ", ky,"kt ", kθ)
    q0[k,:] = [δy*ky;θ_l + δθ*kθ]
    println(q0[k,:])

    θ0 = θ_l + δθ*kθ
    y0 = δy*ky + sin(abs(θ0) + atan(h/w))*sqrt(h^2 + w^2)/2
    x0 = [0;y0;θ0;0;0;0]
    
    initial_errs[k,:] .= [y0; θ0]
    k+=1
end

k 1ky 0kt 0[0.0, -0.7853981633974483]
k 2ky 0kt 1[0.0, -0.7330382858376184]
k 3ky 0kt 2[0.0, -0.6806784082777885]
k 4ky 0kt 3[0.0, -0.6283185307179586]
k 5ky 0kt 4[0.0, -0.5759586531581288]
k 6ky 0kt 5[0.0, -0.5235987755982989]
k 7ky 0kt 6[0.0, -0.47123889803846897]
k 8ky 0kt 7[0.0, -0.4188790204786391]
k 9ky 0kt 8[0.0, -0.3665191429188092]
k 10ky 0kt 9[0.0, -0.3141592653589793]
k 11ky 0kt 10[0.0, -0.26179938779914946]
k 12ky 0kt 11[0.0, -0.20943951023931962]
k 13ky 0kt 12[0.0, -0.15707963267948966]
k 14ky 0kt 13[0.0, -0.10471975511965981]
k 15ky 0kt 14[0.0, -0.05235987755982996]
k 16ky 0kt 15[0.0, 0.0]
k 17ky 0kt 16[0.0, 0.05235987755982985]
k 18ky 0kt 17[0.0, 0.1047197551196597]
k 19ky 0kt 18[0.0, 0.15707963267948966]
k 20ky 0kt 19[0.0, 0.2094395102393195]
k 21ky 0kt 20[0.0, 0.26179938779914935]
k 22ky 0kt 21[0.0, 0.3141592653589793]
k 23ky 0kt 22[0.0, 0.36651914291880905]
k 24ky 0kt 23[0.0, 0.418879020478639]
k 25ky 0kt 24[0.0, 0.47123889803846897]
k 26ky 0kt 25[0.0, 0.5235987755982

k 193ky 6kt 6[0.16000000000000003, -0.47123889803846897]
k 194ky 6kt 7[0.16000000000000003, -0.4188790204786391]
k 195ky 6kt 8[0.16000000000000003, -0.3665191429188092]
k 196ky 6kt 9[0.16000000000000003, -0.3141592653589793]
k 197ky 6kt 10[0.16000000000000003, -0.26179938779914946]
k 198ky 6kt 11[0.16000000000000003, -0.20943951023931962]
k 199ky 6kt 12[0.16000000000000003, -0.15707963267948966]
k 200ky 6kt 13[0.16000000000000003, -0.10471975511965981]
k 201ky 6kt 14[0.16000000000000003, -0.05235987755982996]
k 202ky 6kt 15[0.16000000000000003, 0.0]
k 203ky 6kt 16[0.16000000000000003, 0.05235987755982985]
k 204ky 6kt 17[0.16000000000000003, 0.1047197551196597]
k 205ky 6kt 18[0.16000000000000003, 0.15707963267948966]
k 206ky 6kt 19[0.16000000000000003, 0.2094395102393195]
k 207ky 6kt 20[0.16000000000000003, 0.26179938779914935]
k 208ky 6kt 21[0.16000000000000003, 0.3141592653589793]
k 209ky 6kt 22[0.16000000000000003, 0.36651914291880905]
k 210ky 6kt 23[0.16000000000000003, 0.4188790204

k 376ky 12kt 3[0.32000000000000006, -0.6283185307179586]
k 377ky 12kt 4[0.32000000000000006, -0.5759586531581288]
k 378ky 12kt 5[0.32000000000000006, -0.5235987755982989]
k 379ky 12kt 6[0.32000000000000006, -0.47123889803846897]
k 380ky 12kt 7[0.32000000000000006, -0.4188790204786391]
k 381ky 12kt 8[0.32000000000000006, -0.3665191429188092]
k 382ky 12kt 9[0.32000000000000006, -0.3141592653589793]
k 383ky 12kt 10[0.32000000000000006, -0.26179938779914946]
k 384ky 12kt 11[0.32000000000000006, -0.20943951023931962]
k 385ky 12kt 12[0.32000000000000006, -0.15707963267948966]
k 386ky 12kt 13[0.32000000000000006, -0.10471975511965981]
k 387ky 12kt 14[0.32000000000000006, -0.05235987755982996]
k 388ky 12kt 15[0.32000000000000006, 0.0]
k 389ky 12kt 16[0.32000000000000006, 0.05235987755982985]
k 390ky 12kt 17[0.32000000000000006, 0.1047197551196597]
k 391ky 12kt 18[0.32000000000000006, 0.15707963267948966]
k 392ky 12kt 19[0.32000000000000006, 0.2094395102393195]
k 393ky 12kt 20[0.320000000000000

k 558ky 17kt 30[0.4533333333333334, 0.7853981633974483]
k 559ky 18kt 0[0.4800000000000001, -0.7853981633974483]
k 560ky 18kt 1[0.4800000000000001, -0.7330382858376184]
k 561ky 18kt 2[0.4800000000000001, -0.6806784082777885]
k 562ky 18kt 3[0.4800000000000001, -0.6283185307179586]
k 563ky 18kt 4[0.4800000000000001, -0.5759586531581288]
k 564ky 18kt 5[0.4800000000000001, -0.5235987755982989]
k 565ky 18kt 6[0.4800000000000001, -0.47123889803846897]
k 566ky 18kt 7[0.4800000000000001, -0.4188790204786391]
k 567ky 18kt 8[0.4800000000000001, -0.3665191429188092]
k 568ky 18kt 9[0.4800000000000001, -0.3141592653589793]
k 569ky 18kt 10[0.4800000000000001, -0.26179938779914946]
k 570ky 18kt 11[0.4800000000000001, -0.20943951023931962]
k 571ky 18kt 12[0.4800000000000001, -0.15707963267948966]
k 572ky 18kt 13[0.4800000000000001, -0.10471975511965981]
k 573ky 18kt 14[0.4800000000000001, -0.05235987755982996]
k 574ky 18kt 15[0.4800000000000001, 0.0]
k 575ky 18kt 16[0.4800000000000001, 0.05235987755982

k 748ky 24kt 3[0.6400000000000001, -0.6283185307179586]
k 749ky 24kt 4[0.6400000000000001, -0.5759586531581288]
k 750ky 24kt 5[0.6400000000000001, -0.5235987755982989]
k 751ky 24kt 6[0.6400000000000001, -0.47123889803846897]
k 752ky 24kt 7[0.6400000000000001, -0.4188790204786391]
k 753ky 24kt 8[0.6400000000000001, -0.3665191429188092]
k 754ky 24kt 9[0.6400000000000001, -0.3141592653589793]
k 755ky 24kt 10[0.6400000000000001, -0.26179938779914946]
k 756ky 24kt 11[0.6400000000000001, -0.20943951023931962]
k 757ky 24kt 12[0.6400000000000001, -0.15707963267948966]
k 758ky 24kt 13[0.6400000000000001, -0.10471975511965981]
k 759ky 24kt 14[0.6400000000000001, -0.05235987755982996]
k 760ky 24kt 15[0.6400000000000001, 0.0]
k 761ky 24kt 16[0.6400000000000001, 0.05235987755982985]
k 762ky 24kt 17[0.6400000000000001, 0.1047197551196597]
k 763ky 24kt 18[0.6400000000000001, 0.15707963267948966]
k 764ky 24kt 19[0.6400000000000001, 0.2094395102393195]
k 765ky 24kt 20[0.6400000000000001, 0.261799387799

k 931ky 30kt 0[0.8000000000000002, -0.7853981633974483]
k 932ky 30kt 1[0.8000000000000002, -0.7330382858376184]
k 933ky 30kt 2[0.8000000000000002, -0.6806784082777885]
k 934ky 30kt 3[0.8000000000000002, -0.6283185307179586]
k 935ky 30kt 4[0.8000000000000002, -0.5759586531581288]
k 936ky 30kt 5[0.8000000000000002, -0.5235987755982989]
k 937ky 30kt 6[0.8000000000000002, -0.47123889803846897]
k 938ky 30kt 7[0.8000000000000002, -0.4188790204786391]
k 939ky 30kt 8[0.8000000000000002, -0.3665191429188092]
k 940ky 30kt 9[0.8000000000000002, -0.3141592653589793]
k 941ky 30kt 10[0.8000000000000002, -0.26179938779914946]
k 942ky 30kt 11[0.8000000000000002, -0.20943951023931962]
k 943ky 30kt 12[0.8000000000000002, -0.15707963267948966]
k 944ky 30kt 13[0.8000000000000002, -0.10471975511965981]
k 945ky 30kt 14[0.8000000000000002, -0.05235987755982996]
k 946ky 30kt 15[0.8000000000000002, 0.0]
k 947ky 30kt 16[0.8000000000000002, 0.05235987755982985]
k 948ky 30kt 17[0.8000000000000002, 0.1047197551196

LoadError: UndefVarError: idx_hybrid_success not defined

In [396]:
debug = false
k = 1
# t_sim_start = time()

for ky = 0:16, kθ = 0:(N-1)
    
    if k <= 506
        k+=1
        continue
    end
    
    y0 = initial_errs[k,1]
    θ0 = initial_errs[k,2]
    
    x0 = [0;y0;θ0;0;0;0]
    
    initial_mode = mode(x0, zeros(3))
    
#     t_sim_start = time()
    prob = ODEProblem(dynamics!, x0, (0,2.5), (initial_mode, controller, [0.0], h_control, zeros(3)))
    cb = VectorContinuousCallback(conditions, affect!, affect_neg!, callback_length)
    sol = solve(prob, Tsit5(); callback = cb, abstol=1e-15,reltol=1e-15, adaptive = false, dt=Δt)
    
    print("Sample ", k, " y0 ", y0, ", θ0 ", θ0, ", Simulation status: ", sol.retcode)
     
    u_end = sol.u[end]
    err = norm(u_end[2:4] .- xref[2:4])
    println(", err: ", err)
    errs[k] = err
    
    if string(sol.retcode) != "Success"
        k+=1
        continue
    end
    
    # animation 
    n_plot = length(sol.t)
    # n = Int(floor(0.77/Δt))
    x_plot = zeros(n_plot)
    y_plot = zeros(n_plot)
    θ_plot = zeros(n_plot)
    for k = 1:n_plot
        x_plot[k] = sol.u[k][1]
        y_plot[k] = sol.u[k][2]
        θ_plot[k] = sol.u[k][3]
    end

    anim = @animate for i ∈ 1:n_plot
        plot([-10,5],[0,0], lw = 2, c=:black, xlims=(-2,3), ylims=(-0.5,3))
        plot!(boxshape([x_plot[i],y_plot[i],θ_plot[i]]), aspect_ratio=:equal, c=:gray, opacity=.5, legend=false)
    end
    
    gif(anim, "figs_sample_new/" *ctrl* "_" * string(k) * ".gif", fps = Int(floor(n_plot/tspan[2])));
    
    k += 1
    
    if (mod(k, 10) == 0)
        save(ctrl*"_errs_new.jld", "errs", errs)
    end
    
end

save(ctrl*"_errs_new.jld", "errs", errs)


┌ Warning: Problem status INFEASIBLE; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/packages/Convex/uI27T/src/solution.jl:263
┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/figs_sample_new/hybrid_507.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114
┌ Warning: Problem status INFEASIBLE; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/packages/Convex/uI27T/src/solution.jl:263
┌ Warning: Problem status ALMOST_OPTIMAL; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/packages/Convex/uI27T/src/solution.jl:263
┌ Warning: Problem status ALMOST_OPTIMAL; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/packages/Convex/uI27T/src/solution.jl:263
┌ Warning: Problem status ALMOST_OPTIMAL; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/packages/Convex/uI27T/src/solution.jl:263
┌ Warning: Problem status ALMOST_OPTIMAL; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/package

Sample 507 y0 0.9424465197335545, θ0 -0.26179938779914946, Simulation status: Success, err: 10.785576641987417
Sample 508 y0 0.9218815523690688, θ0 -0.20943951023931962, Simulation status: Success, err: 0.016806374728619296
Sample 509 y0 0.8999592354248374, θ0 -0.15707963267948966, Simulation status: Success, err: 10.790758158428316
Sample 510 y0 0.8767396564478029, θ0 -0.10471975511965981, Simulation status: Success, err: 0.0002618389874978561
Sample 511 y0 0.8522864586899683, θ0 -0.05235987755982996, Simulation status: Success, err: 11.494462765836705
Sample 512 y0 0.8266666666666668, θ0 0.0, Simulation status: Success, err: 0.014192055009131295
Sample 513 y0 0.8522864586899682, θ0 0.05235987755982985, Simulation status: Success, err: 0.00013691169409303484
Sample 514 y0 0.8767396564478028, θ0 0.1047197551196597, Simulation status: Success, err: 2.1748882332192117e-5
Sample 515 y0 0.8999592354248374, θ0 0.15707963267948966, Simulation status: Success, err: 9.814045806611807
Sample 51

┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/figs_sample_new/hybrid_520.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114
┌ Warning: Problem status ALMOST_OPTIMAL; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/packages/Convex/uI27T/src/solution.jl:263
┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/figs_sample_new/hybrid_521.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114
┌ Warning: Problem status INFEASIBLE; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/packages/Convex/uI27T/src/solution.jl:263
┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/figs_sample_new/hybrid_522.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114
┌ Warning: Problem status INFEASIBLE; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/packages/Convex/uI27T/src/solution.jl:263
┌ Info: Saved animation 

In [397]:
save(ctrl*"_errs_new.jld", "errs", errs)


In [398]:
save(ctrl*"_errs_new.jld", "errs", errs[1:k-1])


In [440]:
save(ctrl*"_errs_new.jld", "errs", hybrid_errs)


In [405]:
hybrid_errs = load("hybrid_errs_new.jld","errs");
lqr_errs = load("lqr_errs_new.jld", "errs");
hybrid_errs[hybrid_errs.==0.0] .= NaN
lqr_errs[lqr_errs.==0.0] .= NaN

1-element view(::Vector{Float64}, [264]) with eltype Float64:
 NaN

In [446]:
hybrid_idx_success = (hybrid_errs.<= 0.4)
hybrid_idx_fail = hybrid_errs .> 0.4

q0_hybrid = q0[1:length(hybrid_errs),:]

scatter(q0_hybrid[hybrid_idx_success,1], q0_hybrid[hybrid_idx_success,2],m=:circle, c=:green,markersize=4, markerstrokewidth=0,
        xlabel = "Δheight", ylabel = "Δθ", label = "success", ylim = (-0.85, 0.3))
# scatter!([q0_hybrid[68,1]], [q0_hybrid[68,2]],m=:circle, c=:yellow,markersize=2, markerstrokewidth=0)
scatter!(q0_hybrid[hybrid_idx_fail,1], q0_hybrid[hybrid_idx_fail,2],m=:circle, c=:red,markersize=3, markerstrokewidth=0, label = "fail")
# scatter!(q0[lqr_idx_nan,1], q0[lqr_idx_nan,2],m=:circle, c=:gray,markersize=2, markerstrokewidth=0, label = "nan")
savefig("hybrid_trials.pdf")

In [444]:
q0

961×2 Matrix{Float64}:
 0.0  -0.785398
 0.0  -0.733038
 0.0  -0.680678
 0.0  -0.628319
 0.0  -0.575959
 0.0  -0.523599
 0.0  -0.471239
 0.0  -0.418879
 0.0  -0.366519
 0.0  -0.314159
 0.0  -0.261799
 0.0  -0.20944
 0.0  -0.15708
 ⋮    
 0.8   0.20944
 0.8   0.261799
 0.8   0.314159
 0.8   0.366519
 0.8   0.418879
 0.8   0.471239
 0.8   0.523599
 0.8   0.575959
 0.8   0.628319
 0.8   0.680678
 0.8   0.733038
 0.8   0.785398

In [447]:
lqr_idx_success = (lqr_errs.<= 0.2)
lqr_idx_fail = lqr_errs .> 0.2

q0_lqr = q0[1:length(lqr_errs),:]

scatter(q0_lqr[lqr_idx_success,1], q0_lqr[lqr_idx_success,2],m=:circle, c=:green,markersize=4, markerstrokewidth=0,
        xlabel = "Δy", ylabel = "Δθ", label = "success", ylim = (-0.85, 0.3))
scatter!(q0_lqr[lqr_idx_fail,1], q0_lqr[lqr_idx_fail,2],m=:circle, c=:red,markersize=3, markerstrokewidth=0, label = "fail")
# scatter!(q0[lqr_idx_nan,1], q0[lqr_idx_nan,2],m=:circle, c=:gray,markersize=2, markerstrokewidth=0, label = "nan")
savefig("lqr_trials.pdf")

In [ ]:
errs[errs.==0.0] .= NaN

In [ ]:
sum(errs[528:558].==0.0)

In [ ]:
println((errs[528:558].<0.2) .& (errs[528:558].> 0))

## debug

In [381]:
debug = true
kk = 168
θ0 = initial_errs[kk, 2]
y0 = initial_errs[kk, 1]
x0 = [0;y0;θ0;0;0;0]
# x0 = [0; h*2.5;0.7;0;0;0]

initial_mode = mode(x0, zeros(3))

# initial_mode = [0 1 0 1]

prob = ODEProblem(dynamics!, x0, tspan, (initial_mode, controller, [0.0], h_control, zeros(3)))
cb = VectorContinuousCallback(conditions, affect!, affect_neg!, callback_length)
sol = solve(prob, Tsit5(); callback = cb, abstol=1e-15,reltol=1e-15, adaptive = false, dt=Δt)

n_plot = length(sol.t)
# n = Int(floor(0.77/Δt))
x_plot = zeros(n_plot)
y_plot = zeros(n_plot)
θ_plot = zeros(n_plot)
for k = 1:n_plot
    x_plot[k] = sol.u[k][1]
    y_plot[k] = sol.u[k][2]
    θ_plot[k] = sol.u[k][3]
end

anim = @animate for i ∈ 1:n_plot
    plot([-10,5],[0,0], lw = 2, c=:black, xlims=(-2,3), ylims=(-0.5,3))
    plot!(boxshape([x_plot[i],y_plot[i],θ_plot[i]]), aspect_ratio=:equal, c=:gray, opacity=.5, legend=false)
end

gif(anim, "anim.gif", fps = Int(floor(n_plot/tspan[2])));
    

Dynamics evalutation 0.0x [0.0, 0.6066259020915039, -0.15707963267948966, 0.0, 0.0, 0.0]
Dynamics evalutation 0.00805x [0.0, 0.6066259020915039, -0.15707963267948966, 0.0, -0.0789705, 0.0]
Dynamics evalutation 0.01635x [0.0, 0.605301248336276, -0.15707963267948966, 0.0, -0.16039350000000002, 0.0]
Dynamics evalutation 0.045000000000000005x [0.0, 0.5967521518147844, -0.15707963267948966, 0.0, -0.44145000000000006, 0.0]
Dynamics evalutation 0.049001277045225486x [0.0, 0.5949464704388748, -0.15707963267948966, 0.0, -0.48070252781366274, 0.0]
Dynamics evalutation 0.05x [0.0, 0.5944687032371622, -0.15707963267948966, 0.0, -0.4904999999999999, 0.0]
Dynamics evalutation 0.05x [0.0, 0.5943634020915041, -0.15707963267948966, 0.0, -0.4904999999999999, 0.0]
Dynamics evalutation 0.058050000000000004x [0.0, 0.5904148770915041, -0.15707963267948966, 0.0, -0.5694704999999999, 0.0]
Control evaluation [3.166073562177102, -0.5584609647222365, 0.27923048236111825]
Dynamics evalutation 0.06635x [0.0, 0.585

Dynamics evalutation 0.23993802804365305x [0.026441189803020283, 0.4182101225664648, -0.03594457248790639, 0.21040523270639508, -0.6448349384672233, 1.322048372902708]
Dynamics evalutation 0.2409367509984276x [0.02665818089332277, 0.41749617612567197, -0.03436490271781016, 0.21458317814040448, -0.6417879758634973, 1.3132174956804865]
Dynamics evalutation 0.2409367509984276x [0.02664799053981343, 0.41677895589595615, -0.03402522111038202, 0.21775612716261714, -0.7722911957057955, 1.5890878398015966]
Dynamics evalutation 0.1923071418116179x [0.021069847065873632, 0.45170626432478694, -0.1083097341008248, 0.01455979121329102, -0.7344732402326888, 1.6183628986708538]
Dynamics evalutation 0.19372009134571477x [0.02109842146036816, 0.4506715355073301, -0.10603502264731698, 0.020523207682013225, -0.7465935156299799, 1.6414526512698795]
Dynamics evalutation 0.19859732076160333x [0.021253667515642567, 0.44658710501716214, -0.09707961842421739, 0.03923817913446628, -0.641015430532712, 1.39198496

Dynamics evalutation 0.25585787723946657x [0.03056968271936125, 0.4000000062553775, 1.2510509598332264e-8, 0.24103315701438477, -0.30157643847940074, -0.6031528997551229]
Dynamics evalutation 0.25585787723946657x [0.030569684945380197, 0.4000000008357234, 1.6714467315242393e-9, 0.24074871400483933, -0.3008829196516673, -0.6017658401095023]
Dynamics evalutation 0.25585787723946657x [0.030569685297616812, 0.39999999999956454, 8.709766290673252e-13, 0.09029749511193455, -1.3877787807814457e-17, 2.7755575615609574e-17]
Dynamics evalutation 0.26390787723946657x [0.03129658013326789, 0.39999999999956454, 8.709768524997089e-13, 0.11007968546487581, -2.1027624086400466e-17, 4.205524817277163e-17]
Dynamics evalutation 0.27220787723946654x [0.032377876452030906, 0.39999999999956454, 8.709773227341626e-13, 0.13047622955548605, -2.839950496991151e-17, 5.679900993978344e-17]
Dynamics evalutation 0.30085787723946655x [0.0371064570373125, 0.39999999999956454, 8.709796659687207e-13, 0.2008811678923515

Dynamics evalutation 0.7058578772394667x [0.3074832534526415, 0.39999999999956454, 8.710033576496837e-13, 0.9275171086626838, -1.0606860332002053e-18, 2.121372066398971e-18]
Dynamics evalutation 0.7139078772394667x [0.3149497661773761, 0.39999999999956454, 8.710033747267288e-13, 0.9314266907472948, -4.63560417249321e-18, 9.271208344979998e-18]
Dynamics evalutation 0.7222078772394667x [0.3227137376372987, 0.39999999999956454, 8.710035122657051e-13, 0.9354576884245956, -8.32154461424873e-18, 1.6643089228485902e-17]
Dynamics evalutation 0.7508578772394667x [0.3497103417897279, 0.39999999999956454, 8.710043470616731e-13, 0.9493719153468448, -2.1044700476453023e-17, 4.208940095287675e-17]
Dynamics evalutation 0.7548591542846922x [0.35351098833440564, 0.39999999999956454, 8.710045190327687e-13, 0.9513151850426932, -2.282162443785842e-17, 4.5643248875685037e-17]
Dynamics evalutation 0.7558578772394667x [0.35446097371935265, 0.39999999999956454, 8.710045644075497e-13, 0.9518002272006405, -2.32

Dynamics evalutation 0.8861280579514059x [0.47891044814976436, 0.3981840720886979, 0.003626602878652992, 1.0343842091873128, -0.1350365419342814, 0.26929355984767944]
Dynamics evalutation 0.8941780579514059x [0.48723724103372223, 0.39709702792612694, 0.005794416035426812, 1.054559884527269, -0.17717976791372197, 0.352873171619207]
Dynamics evalutation 0.9024780579514059x [0.4961610574093794, 0.39526931277426514, 0.009431519729307992, 1.0755737111162558, -0.22105494469245118, 0.43890505182577794]
Dynamics evalutation 0.931128057951406x [0.5280264679667879, 0.38674592716780176, 0.0261635731190541, 1.1500859562080001, -0.37645673019662035, 0.73591817702535]
Control evaluation [4.516374036710305, -0.1943563688955295, 0.09717818444776476]
Dynamics evalutation 0.9351293349966314x [0.5326462969082634, 0.38520269455086803, 0.029132431669391737, 1.1623860094990803, -0.3984834838346256, 0.7757882959631341]
Dynamics evalutation 0.936128057951406x [0.5338041597066453, 0.3848024944063207, 0.0299048

...

Dynamics evalutation 3.014299253866673x [4.388805744218333, -5.50438547232237, 9.689394699561962, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.0183005309118984x [4.396465910244375, -5.516661476198935, 9.709522093873959, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.0192992538666727x [4.398377895733288, -5.519725579665366, 9.714545912642812, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.0192992538666727x [4.398377895733288, -5.519725579665366, 9.714545912642812, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.0273492538666726x [4.413789059672368, -5.54442315248759, 9.755039365702979, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.035649253866673x [4.4296788311871955, -5.569887730676964, 9.796790379417189, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.0642992538666727x [4.48452725936

Dynamics evalutation 3.383246132173174x [5.095128828242324, -6.636322415739405, 11.545287009922166, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.391546132173174x [5.111018599757152, -6.6617869939287795, 11.587038023636376, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.420196132173174x [5.165867027937852, -6.749685809004149, 11.731154474589642, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.4241974092183995x [5.173527193963894, -6.761961812880715, 11.75128186890164, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.425196132173174x [5.175439179452807, -6.765025916347145, 11.756305687670492, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.425196132173174x [5.175439179452807, -6.765025916347145, 11.756305687670492, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.4332461321731738x [5.1908503

Dynamics evalutation 3.65344033773195x [5.6123968030566935, -7.465284039084861, 12.904429417366085, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.6574416147771758x [5.620056969082736, -7.477560042961427, 12.924556811678082, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.65844033773195x [5.621968954571649, -7.480624146427857, 12.929580630446935, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.65844033773195x [5.621968954571649, -7.480624146427857, 12.929580630446935, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.6084403377325076x [5.526247439423154, -7.327223072999604, 12.678068499641247, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.6084403377330827x [5.526247439424255, -7.327223073001367, 12.678068499644137, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.6084403377350664x [5.5262474

Dynamics evalutation 3.896903622633353x [6.0784902933377305, -8.212234623978071, 14.129108808549828, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.9049536226333528x [6.09390145727681, -8.236932196800296, 14.169602261609995, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.913253622633353x [6.109791228791638, -8.26239677498967, 14.211353275324205, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.941903622633353x [6.164639656972337, -8.35029559006504, 14.355469726277471, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.9459048996785784x [6.172299822998379, -8.362571593941604, 14.375597120589468, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.9469036226333527x [6.174211808487293, -8.365635697408036, 14.380620939358321, 1.9144303029912515, -3.068021468599286, 5.03024261616986]
Dynamics evalutation 3.9469036226333527x [6.1742118

┌ Warning: Problem status INFEASIBLE; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/packages/Convex/uI27T/src/solution.jl:263
┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/anim.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114


In [327]:
x_test = [-0.0002805237875265441, 0.4000344925886284, 6.898708099831898e-5, -5.080663373451255e-8, 3.4966772008016014e-8, 5.507361018459586e-8];
u_test = [7.1496060413103875, 1.122229993246347, -0.5611149966231735];
mode_test = [1, 1, 0, 0];

In [331]:
solveEOM(x_test, mode_test, controller(x_test, mode_test))

([-1.1059175783145747e-5, 4.433299301220828e-6, 1.1364008305713749e-5], [1.2483912463850115, 6.513180483841806, -10.115617030751897, 6.5139644575361135])

In [293]:
guard_conditions(x_test, [0.0063395758138329395, 0.5141264235600459, -0.1558455371217486, 0.17734481179485495, -1.0336645675969758, 0.14218919068003474]mode_test, u_test)

LoadError: MethodError: no method matching *(::Vector{Float64}, ::Vector{Int64})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  *([91m::StridedMatrix{T}[39m, ::StridedVector{S}) where {T<:Union{Float32, Float64, ComplexF32, ComplexF64}, S<:Real} at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/matmul.jl:44
[0m  *(::StridedVecOrMat{T} where T, [91m::Adjoint{var"#s832", var"#s831"} where {var"#s832", var"#s831"<:LinearAlgebra.LQPackedQ}[39m) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/lq.jl:254
[0m  ...

In [294]:
compute_IV([0.028108702515010013, 0.40000054521948536, 1.0652178176458193e-6, 0.34974013249348335, -0.9020617780252725, 1.8041907001906015])

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [329]:
compute_a(x_test)

2-element Vector{Float64}:
 3.638853541404474e-18
 6.898708094360187e-5

In [284]:
initial_mode = mode(x0, zeros(3))

4-element Vector{Int64}:
 0
 0
 0
 0

In [260]:
compute_a(x0)

2-element Vector{Float64}:
 0.2879116908177594
 0.08000000000000002